In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt
import csv
import pickle 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
#open glycoscored df
#this example is adding noglycogenes for TIL model
with open('TILglyconorm_split.pkl', 'rb') as f:
   dict_for_model = pickle.load(f)

In [5]:
#get ranked nonglycogenes from best model
with open('fullTILfeats_1.pkl', 'rb') as f:
   extragenes = pickle.load(f)

feature_list = extragenes['feature'].tolist()

In [8]:
#get glcyogene list, doesn't matter if it is ranked or not
with open('topTILglycogenes.pkl', 'rb') as f:
   ranked_glycogenes = pickle.load(f)

In [15]:
#filter glycogenes out of top predictive gene list from full transcriptome
nonglyco = [x for x in feature_list if x not in ranked_glycogenes]

In [10]:
gene_list = ranked_glycogenes
tag_list = ['Biotin', 'Type', 'PHA-L']
base_list = tag_list + gene_list

In [17]:
'''
Model evaluation
'''

def model_evaluation(model, x, y):
    
    model_predict = model.predict(x)
    model_predict_prob = model.predict_proba(x)
    
    high = f"{100*(model.score(x[y==1], y[y==1])):>4f}%"
    low = f'{100*(model.score(x[y==0], y[y==0])):>4f}%'
    total = f'{100*(model.score(x, y)):>4f}%'

    avg_loss = log_loss(y, model_predict_prob)
    roc_auc = roc_auc_score(y, model_predict)
    f1 = f1_score(y, model_predict)
    return high, low, total, avg_loss, roc_auc, f1

In [18]:
'''
Generate training, validation, and test set from original full df using scikit learn 
saves best estimator model as a pickle

Input: 
- dataframe with PHA-L column (binary) and transcripts

Output: 
- high, low, total accuracy of best estimator model 
'''
def train_and_evaluate(data, random_number):
    #y: PHA-L score array
    y = data['PHA-L'].values 

    #X: glycogene transcript data array
    x = data.iloc[:, 3:].values   

    # Split training, validation and test set
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.1, random_state=random_number, stratify=y)

    encoder = LabelEncoder()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.fit_transform(y_test)
    
    ''''Use RandomSearchCV to optimize hyperparameters'''
    #takes a while to run!!
    model = RandomForestClassifier()

    model_random = RandomizedSearchCV(estimator = model, param_distributions = random_grid, 
                                      n_iter = 20, cv = 5, verbose=5, random_state=random_number, n_jobs = -1)

    model_random.fit(x_train, y_train)
    
    mod = model_random.best_estimator_
    
    high, low, total, avg_loss, roc_auc, f1 = model_evaluation(mod, x_test, y_test)
    
    return high, low, total, avg_loss, roc_auc, f1



In [20]:
'''
Iteratively add 5 nonglycogenes at a time to 3 models to see how auROC changes
'''

whole_transcript_AUC_arr = [0.896357, 0.893189, 0.893509] #will change for LN data

# Parameters for grid search
# Number of trees in random forest
n_estimators = [int(x) for x in np.arange(200, 800, step=100)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.arange(10, 50, step=10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               }

gene_list = base_list

top_feats_whole_transcriptome_list = nonglyco

random_number = 42

counter = 1

cols_to_add = [] #starts empty, populates with 5 genes each iteration of the while loop

all_cols_for_df = base_list #initialize columnn selection for first df
#make df for model evaluation
master_df = pd.DataFrame(columns=['row_name', 'input', 'high', 'low', 'total', 'avg_loss', 'roc_auc', 'f1', 'genes']) #initialize df to compile results
#make df to store p vals
pval_df = pd.DataFrame(columns=['count', 'statistic', 'pval', 'genes'])
                       
while True: 
    for key, df in dict_for_model.items():
        high, low, total, avg_loss, roc_auc, f1 = train_and_evaluate(df[all_cols_for_df], 
                                                                     42)
        new_row = {'row_name': f"results{counter}_{key}",
                                        'input': random_number, 'high': high, 'low': low,
                                        'total': total,'avg_loss': avg_loss, 'roc_auc': roc_auc, 'f1': f1, 
                                        'genes': top_feats_whole_transcriptome_list[:5] }
        master_df.loc[len(master_df)] = new_row
    temp_array = master_df['roc_auc'].tail(3) #get array of the three roc_auc results from most recent evaluation
    t_statistic, p_value = stats.ttest_rel(whole_transcript_AUC_arr, temp_array)
    print(counter) #see how code is progressing while running
    n_row = {'count': counter, 'statistic': t_statistic, 'pval': p_value, 'genes': top_feats_whole_transcriptome_list[:5]}
    pval_df.loc[len(pval_df)] = n_row
    print('length of genes left:')#see how code is progressing while running
    print(len(top_feats_whole_transcriptome_list))
    
    #break if run out of extra genes to add
    if len(top_feats_whole_transcriptome_list) == 0:  #change this number depending on how many genes you want to add
        break
    if  len(top_feats_whole_transcriptome_list) != 0:
        cols_to_add.extend(top_feats_whole_transcriptome_list[:5])
        top_feats_whole_transcriptome_list = top_feats_whole_transcriptome_list[5:]
        all_cols_for_df = gene_list + cols_to_add
        print(top_feats_whole_transcriptome_list[:5])
        counter += 1
    else:
        break
    
with open('master_df_addnonglyco.vals.pkl', 'wb') as f:
    pickle.dump(master_df, f)
          
with open('master_df_addnonglyco.stats.pkl', 'wb') as f:
    pickle.dump(pval_df, f)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
1
length of genes left:
30802
['Tmsb4x', 'Lcp1', 'AW112010', 'Tpt1', 'Itm2b']
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
2
length of genes left:
30797
['Ubl5', 'Rpl32', 'Tnfrsf9', 'Atp5l', 'Fau']
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
3
length of genes left:
30792
['Ctla4', 'S100a10', 'Fxyd5', 'Cmtm7', 'Rps15a']
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
4
length of genes left:


[CV 3/5] END max_depth=30, min_samples_leaf=1, min_samples_split=5, n_estimators=300;, score=0.840 total time=   1.9s
[CV 5/5] END max_depth=40, min_samples_leaf=1, min_samples_split=2, n_estimators=300;, score=0.852 total time=   2.2s
[CV 5/5] END max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=200;, score=0.847 total time=   0.9s
[CV 1/5] END max_depth=40, min_samples_leaf=2, min_samples_split=2, n_estimators=700;, score=0.842 total time=   3.4s
[CV 1/5] END max_depth=20, min_samples_leaf=2, min_samples_split=5, n_estimators=400;, score=0.833 total time=   1.5s
[CV 3/5] END max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=400;, score=0.830 total time=   1.7s
[CV 4/5] END max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=600;, score=0.845 total time=   2.4s
[CV 5/5] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=700;, score=0.830 total time=   2.5s
[CV 2/5] END max_depth=20, min_samples_leaf=2, min_sampl

[CV 3/5] END max_depth=20, min_samples_leaf=2, min_samples_split=5, n_estimators=700;, score=0.835 total time=   3.7s
[CV 1/5] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=600;, score=0.835 total time=   2.7s
[CV 4/5] END max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=400;, score=0.857 total time=   1.8s
[CV 1/5] END max_depth=None, min_samples_leaf=2, min_samples_split=2, n_estimators=300;, score=0.838 total time=   1.3s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=400;, score=0.837 total time=   1.8s
[CV 3/5] END max_depth=30, min_samples_leaf=1, min_samples_split=5, n_estimators=300;, score=0.833 total time=   1.9s
[CV 5/5] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=600;, score=0.833 total time=   2.5s
[CV 2/5] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=700;, score=0.838 total time=   2.4s
[CV 3/5] END max_depth=10, min_samples_leaf=1, min_s

[CV 1/5] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=600;, score=0.835 total time=   2.3s
[CV 5/5] END max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=600;, score=0.840 total time=   3.3s
[CV 2/5] END max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=400;, score=0.832 total time=   1.6s
[CV 5/5] END max_depth=40, min_samples_leaf=2, min_samples_split=2, n_estimators=700;, score=0.842 total time=   2.9s
[CV 1/5] END max_depth=20, min_samples_leaf=2, min_samples_split=5, n_estimators=700;, score=0.832 total time=   2.7s
[CV 3/5] END max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=600;, score=0.828 total time=   2.5s
[CV 2/5] END max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=200;, score=0.840 total time=   0.9s
[CV 3/5] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=700;, score=0.825 total time=   2.6s
[CV 5/5] END max_depth=10, min_samples_leaf=1, min_sampl

[CV 4/5] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=600;, score=0.849 total time=   3.1s
[CV 5/5] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=600;, score=0.852 total time=   2.0s
[CV 1/5] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=700;, score=0.837 total time=   2.6s
[CV 2/5] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.823 total time=   0.7s
[CV 4/5] END max_depth=20, min_samples_leaf=2, min_samples_split=5, n_estimators=400;, score=0.855 total time=   1.5s
[CV 1/5] END max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=600;, score=0.830 total time=   2.6s
[CV 4/5] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=700;, score=0.847 total time=   3.5s
[CV 1/5] END max_depth=40, min_samples_leaf=2, min_samples_split=5, n_estimators=300;, score=0.830 total time=   1.2s
[CV 3/5] END max_depth=40, min_samples_leaf=2, min_sam

...

[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=700;, score=0.896 total time=  12.5s
[CV 4/5] END max_depth=40, min_samples_leaf=2, min_samples_split=2, n_estimators=700;, score=0.913 total time=  10.2s
[CV 5/5] END max_depth=20, min_samples_leaf=2, min_samples_split=5, n_estimators=400;, score=0.894 total time=   5.6s
[CV 2/5] END max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=600;, score=0.910 total time=   9.2s
[CV 4/5] END max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=200;, score=0.910 total time=   3.1s
[CV 5/5] END max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=200;, score=0.896 total time=   2.9s
[CV 2/5] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.910 total time=   2.7s
[CV 2/5] END max_depth=20, min_samples_leaf=2, min_samples_split=5, n_estimators=700;, score=0.927 total time=   9.5s
[CV 4/5] END max_depth=30, min_samples_leaf=2, min_sam

[CV 3/5] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=700;, score=0.893 total time=   8.6s
[CV 5/5] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.901 total time=   2.5s
[CV 2/5] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=600;, score=0.915 total time=   9.3s
[CV 5/5] END max_depth=40, min_samples_leaf=1, min_samples_split=2, n_estimators=300;, score=0.894 total time=   5.9s
[CV 2/5] END max_depth=30, min_samples_leaf=2, min_samples_split=2, n_estimators=400;, score=0.910 total time=   6.4s
[CV 5/5] END max_depth=40, min_samples_leaf=2, min_samples_split=5, n_estimators=300;, score=0.889 total time=   4.6s
[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=400;, score=0.920 total time=   5.8s
[CV 3/5] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=600;, score=0.886 total time=   8.1s
[CV 4/5] END max_depth=40, min_samples_leaf=1, min_sam

exception calling callback for <Future at 0x7ffa2f870310 state=finished raised BrokenProcessPool>
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/home/jessiemarieille/miniconda3/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py", line 391, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/home/jessiemarieille/miniconda3/lib/python3.10/multiprocessing/queues.py", line 108, in get
    if not self._rlock.acquire(block, timeout):
KeyboardInterrupt
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/jessiemarieille/miniconda3/lib/python3.10/site-packages/joblib/externals/loky/_base.py", line 26, in _invoke_callbacks
    callback(self)
  File "/home/jessiemarieille/miniconda3/lib/python3.10/site-packages/joblib/parallel.py", line 385, in __call__
    self.parallel.dispatch_next()
  File "/home/jessiemariei

[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=700;, score=0.929 total time=  10.3s
[CV 3/5] END max_depth=40, min_samples_leaf=2, min_samples_split=2, n_estimators=700;, score=0.923 total time=   9.7s
[CV 1/5] END max_depth=20, min_samples_leaf=2, min_samples_split=5, n_estimators=400;, score=0.886 total time=   5.3s
[CV 3/5] END max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=400;, score=0.894 total time=   5.6s
[CV 5/5] END max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=600;, score=0.911 total time=   9.7s
[CV 2/5] END max_depth=40, min_samples_leaf=2, min_samples_split=2, n_estimators=700;, score=0.927 total time=   9.6s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=400;, score=0.889 total time=   5.6s
[CV 3/5] END max_depth=30, min_samples_leaf=1, min_samples_split=5, n_estimators=300;, score=0.896 total time=   6.0s
[CV 4/5] END max_depth=10, min_samples_leaf=1, min_s

KeyboardInterrupt: 